In [2]:
import os
import re
import numpy as np
import open3d as o3d
import math

def parse_foldername(foldername):
    # 解析文件夹名,提取位置坐标和朝向角度
    match = re.match(r"\((-?\d+\.?\d*),\s*(-?\d+\.?\d*),\s*(-?\d+\.?\d*)\),\s*(\d+)", foldername)
    if match:
        x, y, z = float(match.group(1)), float(match.group(2)), float(match.group(3))
        yaw = float(match.group(4))
        return (x, y, z), yaw
    else:
        return None, None

def get_transform_matrix(position, yaw):
    # 根据位置和朝向计算变换矩阵
    yaw_rad = math.radians(yaw)
    cos_yaw, sin_yaw = math.cos(yaw_rad), math.sin(yaw_rad)
    rot_matrix = np.array([[cos_yaw, -sin_yaw, 0, 0],
                           [sin_yaw, cos_yaw, 0, 0],
                           [0, 0, 1, 0],
                           [0, 0, 0, 1]])
    trans_matrix = np.array([[1, 0, 0, position[0]],
                             [0, 1, 0, position[1]],
                             [0, 0, 1, position[2]],
                             [0, 0, 0, 1]])
    return trans_matrix @ rot_matrix

def auto_align_clouds(cloud_dirs):
    # 自动拼接点云的主函数
    clouds = []
    for dir in cloud_dirs:
        position, yaw = parse_foldername(os.path.basename(dir))
        if position is None:
            continue
        
        cloud_file = os.path.join(dir, os.listdir(dir)[0])  # 假设每个文件夹下只有一个文件
        cloud = o3d.io.read_point_cloud(cloud_file)
        transform = get_transform_matrix(position, yaw)
        cloud.transform(transform)
        clouds.append(cloud)
    
    merged_cloud = clouds[0]
    for cloud in clouds[1:]:
        merged_cloud += cloud
    
    o3d.io.write_point_cloud("merged_cloud.pcd", merged_cloud)
    
    return merged_cloud

if __name__ == "__main__":
    # 指定包含点云文件夹的根目录路径
    root_dir = "C:\\Users\\iansy\\Downloads\\PQ512(32)"
    cloud_dirs = [os.path.join(root_dir, d) for d in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, d))]
    merged_cloud = auto_align_clouds(cloud_dirs)
    o3d.visualization.draw_geometries([merged_cloud])

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 3 is different from 4)